# Limpieza de datos y cálculos previos

#### ¿Qué hacemos en este cuaderno?
Ordenar la información en bruto que INEI envió por correo. Esta corresponde principalmente a límites de las *ciudades principales*. 
Lo que haremos es calcular la población de cada parte de las manchas urbanas de las *ciudades principales*.

#### ¿Qué procedimiento seguimos?
Básicamente, intersecciones entre distritos y manchas urbanas.

Trabajamos primero en PyQGIS y luego en Python (usando principalmente GeoPandas).

#### ¿Qué datos obtenemos?
1. Un gdf que tiene partidas las manchas urbanas por distritos y la población que la conforma (*4-ciudades_pob.geojson*). No nos interesa su geometría, sino sus datos.
2. Un gdf que tiene solo las manchas urbanas repartidas por distritos (*3-ciudades_interseccion.geojson*). Aquí si nos interesa su geometría.

In [ ]:
##ESTO ES EN PyQGIS

import os
os.chdir('G:/Mi unidad/Documentos personales/Muertes Covid-19 por Áreas urbanas - Perú/1-Cálculos previos con límites INEI')

#1. Cargamos los .shp enviados por INEI
cp="rawdata/PU_PrincipalesCiudades.shp"
lima="rawdata/PU_LimaMetropolitana.shp"

#iface.addVectorLayer(cp, "Ciudades Principales", "ogr")  #por si queremos visualizarlos
#iface.addVectorLayer(lima, "Lima", "ogr")

#2.Haremos una unión de ambos
unido=processing.run("native:mergevectorlayers", {'LAYERS':[cp,lima],
'CRS':QgsCoordinateReferenceSystem('EPSG:4326'),
'OUTPUT':'data/0-unioncapavectorial.geojson'})

#3.Reproyectamos la nueva capa a WGS84:UTM18S
geom=processing.runAndLoadResults("native:reprojectlayer", 
{'INPUT':unido['OUTPUT'],
'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:32718'),
'OPERATION':'+proj=pipeline +step +proj=unitconvert +xy_in=deg +xy_out=rad +step +proj=utm +zone=18 +south +ellps=WGS84',
'OUTPUT':'data/1-union_reproy.geojson'})

#4.Corregimos la geometría obtenida
#Este paso es por si acaso, en mi experiencia suelen haber errores geométricos luego de hacer
#una unión de capas vectoriales que deben ser corregidos
ciudades=processing.run("native:fixgeometries", 
{'INPUT':geom['OUTPUT'],
'OUTPUT':'data/2-manchasurb_ciudades.geojson'})
#manualmente rellenamos la fila del "CODDPTO" al que pertenece la ciudad de Lima Metropolitana

#5. Ahora cargamos la capa de límites distritales (sacada de GEOGPS-Perú)
distritos="rawdata/LimitesDistritales_18S_.gpkg"
#iface.addVectorLayer(distritos, "Distritos", "ogr") #por si queremos verlo
#Vemos que, en realidad, las únicas columnas que nos interesan de este vector,
#son CODUBIGEO, NOMBDIST, IDPROV y NOMBPROV. Ambas nos permitirán saber, luego de hacer la
#intersección, en qué distrito está cada parte de la mancha urbana que
#conforman las ciudades.

#6. Hacemos la intersección
inters=processing.run("native:intersection", 
{'INPUT': ciudades['OUTPUT'],
'OVERLAY':distritos,
'INPUT_FIELDS':[],
'OVERLAY_FIELDS':['NOMBDIST','CODUBIGEO','IDPROV','NOMBPROV'],
'OVERLAY_FIELDS_PREFIX':'','OUTPUT':'TEMPORARY_OUTPUT'})
#6.1.Le ponemos un ID a cada 'pedacito' de mancha urbana repartido entre
#distritos
ciud_inters=processing.run("native:addautoincrementalfield", 
{'INPUT':inters['OUTPUT'],
'FIELD_NAME':'ID','START':1,
'GROUP_FIELDS':[],'SORT_EXPRESSION':'',
'SORT_ASCENDING':True,'SORT_NULLS_FIRST':False,
'OUTPUT':'data/3-ciudades_interseccion.geojson'})

In [2]:
##AHORA PASAMOS A PYTHON
import geopandas as gpd
import pandas as pd
import numpy as np

In [8]:
#1.A lo obtenido necesitamos añadirle la población. Para ello utilizaremos la población por manzana que viene
#de la cartografía de GeoGPS-Perú (A estas alturas ya sabemos que es información confiable).
#Para hacer menos pesado el archivo obtendremos los centroides de cada manzana.

#1.1.Importamos
gdf=gpd.read_file('rawdata/PeruEnManzanas_INEI2017_GC_18S.gpkg') #esta en UTM18S, igual que los archivos obtenidos
                                                                 #en el paso anterior. Este archivo es el que descargué
                                                                 #de GeoGPS, solo que convertido a UTM18S y en .gpkg

    
#1.2. Convertimos el gdf de manzanas en centroides
centros=gpd.GeoDataFrame(gdf.centroid)
centros.rename(columns={0:'geometry'}, 
                inplace=True)                      #para setear el CRS es necesario que la única columna que tiene
                                                   #se llame 'geometry'
    
centros=centros.set_crs('EPSG:32718')              #para que tenga el mismo CRS que el gdf al que se unirá

#1.3. Ahora los centros deben 'absorber' los datos de su gdf original
mz = gpd.sjoin(centros, gdf, how="left")
mz.head()

,geometry,index_right,Mz,UBIGEO,AREA,CODCCPP,ZONA,MANZANA_ID,MANZANA_A,T_TOTAL,T_HOMBRES,T_MUJERES,LLAVE_MZS,contacto,descargar,whatsapp,NOMBDEP,NOMBPROV,IDPROV
0,POINT (546598.404 8580848.223),0.0,090201000100100004B,090201,1,0001,00100,004,B,127.0,60.0,67.0,090201000100100004B,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206.0,HUANCAVELICA,ACOBAMBA,0902
1,POINT (537595.600 8588489.910),1.0,090208005300300019,090208,1,0053,00300,019,None,3.0,2.0,1.0,090208005300300019,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206.0,HUANCAVELICA,ACOBAMBA,0902
2,POINT (537959.076 8580748.412),2.0,090203003100000,090203,2,0031,00000,None,None,15.0,8.0,7.0,090203003100000,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206.0,HUANCAVELICA,ACOBAMBA,0902
3,POINT (534681.734 8586405.771),3.0,090203000600200009,090203,1,0006,00200,009,None,5.0,3.0,2.0,090203000600200009,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206.0,HUANCAVELICA,ACOBAMBA,0902
4,POINT (539455.155 8579111.312),4.0,090203005400000,090203,2,0054,00000,None,None,37.0,14.0,23.0,090203005400000,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206.0,HUANCAVELICA,ACOBAMBA,0902


In [20]:
#1.4.Exportamos
mz=mz[['geometry','Mz','UBIGEO','T_TOTAL']] #nos quedamos con las columnas que nos interesan
#mz.to_file("data/0-manzanas_centros.geojson",driver='GeoJSON') #UTM 18S 

In [21]:
#2.Hacemos el spatial join de los centroides de las manzanas (que tienen info de población)
#con las manchas urbanas distribuidas por 'pedacitos'

#mz=gpd.read_file('data/0-manzanas_centros.geojson')
#manchas=gpd.read_file('data/3-ciudades_interseccion.geojson')

intento1=gpd.sjoin(mz,manchas,how="left", op="within")   #por defecto, 'sjoin' utiliza la opción 'intersect'
                                                         #sin embargo, al usar 'within' los resultados pueden ser hasta
                                                         #10 veces más rápido
intento1.head()

,geometry,Mz,UBIGEO,T_TOTAL,index_right,CODDPTO,NOMBDPTO,CIUDAD,layer,path,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
0,POINT (546598.404 8580848.223),090201000100100004B,090201,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (537595.600 8588489.910),090208005300300019,090208,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (537959.076 8580748.412),090203003100000,090203,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (534681.734 8586405.771),090203000600200009,090203,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (539455.155 8579111.312),090203005400000,090203,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
len(intento1)

483255

In [25]:
#3.Filtramos solo aquellas columnas que estén dentro de las manchas urbanas de las ciudades principales
filtro=intento1['ID']>=0
intento2 = intento1[filtro]

intento2.to_file("data/4-ciudades_pob.geojson",
                driver='GeoJSON')                        #guardamos
intento2.head()

,geometry,Mz,UBIGEO,T_TOTAL,index_right,CODDPTO,NOMBDPTO,CIUDAD,layer,path,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
8165,POINT (907455.505 8420317.044),080601000100200021,080601,78.0,188.0,08,CUSCO,SICUANI,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SICUANI,80601.0,0806,CANCHIS,189.0
8167,POINT (907472.434 8420576.111),080601000100100056J,080601,30.0,188.0,08,CUSCO,SICUANI,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SICUANI,80601.0,0806,CANCHIS,189.0
8170,POINT (907378.489 8417327.332),080601000101000004,080601,90.0,188.0,08,CUSCO,SICUANI,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SICUANI,80601.0,0806,CANCHIS,189.0
8171,POINT (907316.777 8420296.208),080601000100200024,080601,110.0,188.0,08,CUSCO,SICUANI,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SICUANI,80601.0,0806,CANCHIS,189.0
8172,POINT (907345.533 8418036.236),080601000100600003,080601,49.0,188.0,08,CUSCO,SICUANI,PU_PrincipalesCiudades,rawdata/PU_PrincipalesCiudades.shp,SICUANI,80601.0,0806,CANCHIS,189.0


In [24]:
len(intento2)

254599